In [79]:
import pandas as pd
import numpy as np
import re           #useful for searching words in a text or documents
from nltk.corpus import stopwords # words that don't add much value to the text or data like was were is am etc
from nltk.stem.porter import PorterStemmer  # nltk = natural language tool kit used for stemming process
from sklearn.feature_extraction.text import TfidfVectorizer # used to convert text into feature vectors
from sklearn.model_selection import train_test_split # used to train data by splitting
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [80]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

## Data pre-processing

In [82]:
df = pd.read_csv('fake_news_dataset.csv')

In [83]:
df.head()

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake


In [84]:
df.shape

(20000, 7)

In [85]:
df['label'] = df['label'].map({'real':0,'fake':1})

In [86]:
df.head()

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,0
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,1
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,1
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,1
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,1


In [87]:
df.isnull().sum()

title          0
text           0
date           0
source      1000
author      1000
category       0
label          0
dtype: int64

In [88]:
df = df.fillna('') # replacing null value with empty string

In [89]:

df = df.drop_duplicates()
  



In [90]:
df['content'] = df['author']+' '+df['title'] # merging the two column

In [91]:
df = df[df['content'].str.strip() != ''] 

In [92]:
print(df['content'])

0                     Paula George Foreign Democrat final.
1          Joseph Hill To offer down resource great point.
2              Julia Robinson Himself church myself carry.
3                Mr. David Foster DDS You unit its should.
4        Austin Walker Billion believe employee summer ...
                               ...                        
19995                         Gary Miles House party born.
19996    Maria Mcbride Though nation people maybe price...
19997     Kristen Franklin Yet exist with experience unit.
19998                  David Wise School wide itself item.
19999        James Peterson Offer chair cover senior born.
Name: content, Length: 20000, dtype: object


In [93]:
df = df.drop(['date','source','category'],axis=1)

In [94]:
# seperating data and label
x = df.drop(columns='label',axis=1)
y = df['label']
print(x)
print(y)

                                       title  \
0                    Foreign Democrat final.   
1        To offer down resource great point.   
2               Himself church myself carry.   
3                       You unit its should.   
4       Billion believe employee summer how.   
...                                      ...   
19995                      House party born.   
19996  Though nation people maybe price box.   
19997        Yet exist with experience unit.   
19998               School wide itself item.   
19999         Offer chair cover senior born.   

                                                    text  \
0      more tax development both store agreement lawy...   
1      probably guess western behind likely next inve...   
2      them identify forward present success risk sev...   
3      phone which item yard Republican safe where po...   
4      wonder myself fact difficult course forget exa...   
...                                                  ...   
199

In [95]:
x.shape

(20000, 4)

In [96]:
x.isnull().sum()

title      0
text       0
author     0
content    0
dtype: int64

Stemming :
It is the process of reducing a word to its Root word
Eg : Actor, Actress,acting -> act

In [97]:
port_stem = PorterStemmer()

In [98]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # removing special characters and numbers
    stemmed_content = stemmed_content.lower() # converting to lower case
    stemmed_content = stemmed_content.split() # splitting the words
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # stemming the words and removing stopwords
    return ' '.join(stemmed_content) # joining the words back together

In [99]:
df['content'] = df['content'].apply(stemming)

In [100]:
print(df['content'])

0                      paula georg foreign democrat final
1                   joseph hill offer resourc great point
2                             julia robinson church carri
3                                 mr david foster dd unit
4             austin walker billion believ employe summer
                               ...                       
19995                           gari mile hous parti born
19996    maria mcbride though nation peopl mayb price box
19997              kristen franklin yet exist experi unit
19998                         david wise school wide item
19999         jame peterson offer chair cover senior born
Name: content, Length: 20000, dtype: object


In [101]:
# seperating data and label
x = df['content'].values
y = df['label'].values
print(x)
print(y)

['paula georg foreign democrat final'
 'joseph hill offer resourc great point' 'julia robinson church carri' ...
 'kristen franklin yet exist experi unit' 'david wise school wide item'
 'jame peterson offer chair cover senior born']
[0 1 1 ... 0 1 1]


In [102]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=10000)# Tf=turn frequency, idf=inverse document frequency
vectorizer.fit(x)
x = vectorizer.transform(x)
# changed here

In [103]:
print(x)

  (0, 1915)	0.3230188984775633
  (0, 1919)	0.571944191905508
  (0, 2634)	0.31633034321740966
  (0, 2750)	0.35299855456103546
  (0, 2922)	0.38575629231270886
  (0, 5930)	0.4416535406297843
  (1, 3003)	0.34047540791294395
  (1, 3281)	0.3634236210268278
  (1, 4000)	0.3291477077105655
  (1, 4003)	0.549500072846975
  (1, 5681)	0.3391462562106277
  (1, 6551)	0.3395858135219534
  (1, 7744)	0.33372521164763813
  (2, 1108)	0.36865236657763756
  (2, 1355)	0.3738760372037231
  (2, 4056)	0.463176331359564
  (2, 4060)	0.6026092952351498
  (2, 7943)	0.3829384608341644
  (3, 1775)	0.2806133444664392
  (3, 1786)	0.5168731180696984
  (3, 1843)	0.3378791128160659
  (3, 2781)	0.3842313958445138
  (3, 5370)	0.2607369501376606
  (3, 5386)	0.46193396110860874
  (3, 9463)	0.3330698555884235
  :	:
  (19996, 4928)	0.44657233844084904
  (19996, 5480)	0.3011404108931225
  (19996, 5949)	0.338246234612171
  (19996, 6991)	0.31677886528565996
  (19996, 9195)	0.3368318421904574
  (19997, 2490)	0.37503712327607314
  (

Spliting dataset for training and testing

In [104]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

Training the model : Logistic Regression

In [124]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation 
Accuracy Score

In [125]:
# accuracy score on the training data
# binary classification problems logistic regression is best model
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [126]:
print("Accuracy score of the training data : ", training_data_accuracy)

Accuracy score of the training data :  0.7163125


In [127]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [128]:
print("Accuracy score of the test data : ", testing_data_accuracy)

Accuracy score of the test data :  0.49675
